In [1]:
from sqlalchemy import create_engine
import requests
import pandas as pd
from datetime import datetime, timedelta
from time import sleep

# 創建與SQLite連接的引擎
engine = create_engine('sqlite:///stock_data.db')

def fetch_data(url, max_retries=5, delay=2):
    for attempt in range(max_retries):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"HTTP Error {response.status_code} on attempt {attempt + 1}")
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}, attempt {attempt + 1}")
        sleep(delay)  # 等待一段時間後再次嘗試
    print("Max retries reached, moving to next item.")
    return None

# 讀取股票代碼
with open('stock_id.txt', 'r', encoding='utf-8-sig') as file:
    stock_code = file.read().splitlines()

# 爬取過去10年的數據
end_date = datetime.now()
start_date = end_date - timedelta(days=365 * 10)  # 往回計算10年

years = range(start_date.year, end_date.year + 1)

for year in years:
    date_list = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq='D').strftime("%Y%m%d").tolist()
    table_name = f'stock_data_{year}'
    
    for day in date_list:
        url = f"https://www.twse.com.tw/exchangeReport/MI_INDEX?response=json&date={day}&type=ALLBUT0999"
        stock_json = fetch_data(url)
        
        if stock_json and stock_json.get('stat') != '很抱歉，沒有符合條件的資料!':
            if 'data9' in stock_json:
                for code in stock_code:
                    for item in stock_json['data9']:
                        if code == item[0]:
                            df_initial = pd.DataFrame([item], columns=['證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌', '漲跌價差', '最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量', '本益比'])
                            df_initial.insert(0, '日期', datetime.strptime(day, "%Y%m%d"))
                            df_initial.to_sql(table_name, con=engine, if_exists='append', index=False)
                            break
        sleep(2) # 爬取每日數據後暫停1秒，可根據需要調整

In [7]:
from sqlalchemy import create_engine
import pandas as pd

# 創建與SQLite資料庫的連接
engine = create_engine('sqlite:///stock_data.db')

# 定義要查詢的表名
table_name = 'stock_data_2016'

# 使用pandas的read_sql_table方法讀取整個表格的數據
df = pd.read_sql_table(table_name, con=engine)

# 顯示數據
print(df)

               日期  證券代號  證券名稱        成交股數   成交筆數         成交金額    開盤價    最高價  \
0      2016-01-04  1101    台泥  10,437,306  5,250  275,489,048  27.30  27.30   
1      2016-01-04  1102    亞泥   6,852,789  3,472  179,618,513  27.35  27.35   
2      2016-01-04  1103    嘉泥     133,044     68    1,254,896   9.54   9.54   
3      2016-01-04  1104    環泥     231,453    134    5,076,246  22.20  22.20   
4      2016-01-04  1108    幸福      81,000     42      813,850  10.10  10.10   
...           ...   ...   ...         ...    ...          ...    ...    ...   
202269 2016-12-30  9944    新麗      24,000     17      500,500  20.90  20.90   
202270 2016-12-30  9945   潤泰新   1,957,746    919   72,207,843  36.80  37.00   
202271 2016-12-30  9946  三發地產     189,202     62    2,014,690  10.70  10.75   
202272 2016-12-30  9955    佳龍     156,925     92    2,503,721  15.95  16.15   
202273 2016-12-30  9958   世紀鋼      38,120     20      368,132   9.68   9.75   

          最低價    收盤價                           漲跌  